<a href="https://colab.research.google.com/github/SAMAWELJABALLI/TweetsSentimentAnalysis/blob/main/procecessing_of_processed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import resample
    
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# Load data and set labels from corpus 1 
data = pd.read_csv('/content/data_processed.csv')
print(f'Shape of data is {data.shape}')
# Display 20 random samples
data.sample(20)

Shape of data is (199097, 3)


,ID,text_clean,label
70403,22174,جيد جدااااا,1
121073,16328,سلام,1
145314,14230,brabi saybou rajel yekdem rouho,1
165056,34681,tamthil mamstou w mamrgou,0
182407,52672,ti enti wejhek wahdou choubouhett ritek fi emi...,0
70715,22511,الجزائر دولة عربية مسليمة,1
139139,7832,bel chikh officiel hahaha ouii,1
119427,14682,ya rabi yachfiham,1
170645,40475,brabo,1
30063,30063,NaN,1


# Duplicate Sentence

Most webscrapped data contains duplicated sentence.This function could retrieve unique ones.

In [3]:
def pick_unique_sentence(text):
    line=re.findall(r'(?sm)(^[^\r\n]+$)(?!.*^\1$)', text)
    return line


In [4]:
sentence="ya wa7ch dima a3ml talla\n I thank doctors\nDoctors are working very hard in this pandemic situation\nI thank doctors bravo lik y wa7ch \nya wa7ch dima a3ml talla\n  "
pick_unique_sentence(sentence)

[' I thank doctors',
 'Doctors are working very hard in this pandemic situation',
 'I thank doctors bravo lik y wa7ch ',
 'ya wa7ch dima a3ml talla',
 '  ']

In [5]:
data["text_clean"].fillna("No content", inplace = True)
data['pick_unique']=data['text_clean'].apply(lambda x : pick_unique_sentence(x))
data.sample(20)

,ID,text_clean,label,pick_unique
184941,55284,allah,1,[allah]
70627,22416,orange est le meilleur opérateur en tunisie,1,[orange est le meilleur opérateur en tunisie]
50938,1143,اول تجربه ليا معاهم معامله وطريقه مفيش احسن من...,1,[اول تجربه ليا معاهم معامله وطريقه مفيش احسن م...
31030,31030,rabi yesterna inchallah,1,[rabi yesterna inchallah]
140748,9508,bonne annif si slim ou kol wenti,1,[bonne annif si slim ou kol wenti]
126150,21405,tftfarjo fil vedeo ili diz fil métro mchina feha,1,[tftfarjo fil vedeo ili diz fil métro mchina f...
102819,57324,حذاري تشتروا الكاميرات اللي سعرها او مليون فما...,1,[حذاري تشتروا الكاميرات اللي سعرها او مليون فم...
95170,48978,حاجه زي الفل,1,[حاجه زي الفل]
92002,45549,lun des meilleurs application pour achter online,1,[lun des meilleurs application pour achter onl...
90720,44167,طلبت اردر بقالو شهر مجاش وشركه شحن زباله ال تبعهم,0,[طلبت اردر بقالو شهر مجاش وشركه شحن زباله ال ت...


# remove repeated lettre

In [6]:
def rep(text):
    grp = text.group(0)
    if len(grp) > 1:
        return grp[0:1] # can change the value here on repetition
def unique_char(rep,sentence):
    convert = re.sub(r'(\w)\1+', rep, sentence) 
    return convert


In [7]:
sentence="مااااسط مااااسط bravoooooooooooooo degaage"
unique_char(rep,sentence)

'ماسط ماسط bravo degage'

In [8]:
data['pick_unique']=data['pick_unique'].apply(str)
data['unique_char']=data['pick_unique'].apply(lambda x : unique_char(rep,x))
data.head(20)

,ID,text_clean,label,pick_unique,unique_char
0,0,weldin weldikk bravooooo,1,['weldin weldikk bravooooo'],['weldin weldik bravo']
1,1,bravo amine j aime,1,['bravo amine j aime'],['bravo amine j aime']
2,2,صراحة bravo أحمد الرحموني,1,['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني']
3,3,souut w ziiin lee wlh bch f,1,['souut w ziiin lee wlh bch f'],['sout w zin le wlh bch f']
4,4,anaa,1,['anaa'],['ana']
5,5,bravooooo walid,1,['bravooooo walid'],['bravo walid']
6,6,محترم و نموة علها,1,['محترم و نموة علها'],['محترم و نموة علها']
7,7,ربي يشفيلك بوك وان شاء الله غصرة وتتعدى,1,['ربي يشفيلك بوك وان شاء الله غصرة وتتعدى'],['ربي يشفيلك بوك وان شاء اله غصرة وتعدى']
8,8,bravoooooooooooooooo nabara jnab rak,1,['bravoooooooooooooooo nabara jnab rak'],['bravo nabara jnab rak']
9,9,bravo,1,['bravo'],['bravo']


# 1-grame features extracted from processed_text

In [9]:
def ngrams_top(corpus,ngram_range,n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(ngram_range=ngram_range).fit(corpus.values.astype('U'))
    bag_of_words = vec.transform((corpus).values.astype('U'))
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    total_list=words_freq[:n]
    df=pd.DataFrame(total_list,columns=['text','count'])
    return df

In [10]:
une_gram =ngrams_top(data['unique_char'],(1,1),n=200)
une_gram.values.tolist()


[['de', 12140],
 ['ya', 11274],
 ['rabi', 10949],
 ['في', 10552],
 ['et', 10002],
 ['fi', 9941],
 ['ربي', 9897],
 ['el', 8490],
 ['la', 8483],
 ['من', 8057],
 ['اله', 8048],
 ['le', 7707],
 ['je', 7438],
 ['bravo', 7022],
 ['slim', 6958],
 ['content', 6913],
 ['no', 6826],
 ['pas', 6258],
 ['les', 6200],
 ['aplication', 5402],
 ['pour', 4701],
 ['jumia', 4451],
 ['est', 4396],
 ['على', 4376],
 ['très', 4358],
 ['il', 4342],
 ['des', 4299],
 ['si', 4259],
 ['vous', 4196],
 ['en', 4006],
 ['que', 3848],
 ['alah', 3771],
 ['cest', 3721],
 ['الي', 3720],
 ['kol', 3707],
 ['eli', 3626],
 ['un', 3610],
 ['جدا', 3587],
 ['يا', 3505],
 ['merci', 3439],
 ['une', 3398],
 ['ca', 3383],
 ['bone', 3304],
 ['ما', 3290],
 ['bech', 3225],
 ['ne', 3195],
 ['تونس', 3125],
 ['mais', 3120],
 ['ou', 3066],
 ['ken', 3028],
 ['كل', 3009],
 ['bien', 2957],
 ['bon', 2667],
 ['plus', 2544],
 ['men', 2427],
 ['slouma', 2356],
 ['كان', 2355],
 ['barcha', 2296],
 ['jai', 2291],
 ['لا', 2269],
 ['باش', 2065],
 ['am

In [11]:
noise1gram=set(['wiw','inti','inty','na','je','wé','ena','cete','رجلرجل','eni','ana','انت','ben','عن','al','mes','و','w','mel','wel','jour','come','هو','بعد','inti','الى','se','lkol','fama','bel','فيه','ele','فى','kima','haka','kan','nes','بينا','votre','par','ya','mr','une', 'cest','vous','ont','sont','nous','kif','علي','faire','dans','fel','ala','ana','taw','ye','mon','tout','bravobravo','col','مع','li','ça','de','ya','في','et','fi','el','je','من','les','pour','est','على','il','des','si','vous','en','que','cest','الي','kol','eli','un','يا','une','ما','bech','ou','ken','كل','men','كان','jai','باش','ama','mala','cette','هذا','انا','ان','avec','du','dima','ce','min','ili','ki','enti','fih','tawa','qui','au','ena','fil','suis','wa'])
len(noise1gram)

112

# 2-grame features extracted from processed_text


In [12]:
deux_gram =ngrams_top(data['unique_char'],(2,2),n=100)
deux_gram.values.tolist()


[['no content', 6819],
 ['si slim', 2604],
 ['je suis', 1258],
 ['bone aplication', 1199],
 ['cete aplication', 1142],
 ['شاء اله', 992],
 ['slim riahi', 873],
 ['de la', 869],
 ['ربي يسترنا', 851],
 ['ربي يلطف', 779],
 ['سيدي الوزير', 760],
 ['ya si', 758],
 ['يقدر الخير', 749],
 ['ya slouma', 734],
 ['très bone', 714],
 ['je ne', 711],
 ['ya rabi', 688],
 ['يارب العالمين', 658],
 ['ربي يقدر', 655],
 ['je vous', 642],
 ['en ligne', 628],
 ['bone continuation', 617],
 ['la livraison', 576],
 ['جدا جدا', 574],
 ['que je', 574],
 ['يلطف بينا', 568],
 ['ya slim', 559],
 ['ان شاء', 547],
 ['de livraison', 521],
 ['في تونس', 518],
 ['pas de', 514],
 ['pour les', 497],
 ['ربي يستر', 482],
 ['mise jour', 472],
 ['merci jumia', 449],
 ['et je', 447],
 ['il faut', 436],
 ['عبد الطيف', 432],
 ['en plus', 413],
 ['que vous', 406],
 ['يا رب', 403],
 ['cest une', 403],
 ['جميل جدا', 400],
 ['fi tounes', 391],
 ['ce que', 390],
 ['el kol', 388],
 ['pour le', 386],
 ['امين يارب', 382],
 ['bon courage

# 3-grame features extracted from processed_text


In [13]:
trois_gram =ngrams_top(data['unique_char'],(3,3),n=100)
trois_gram.values.tolist()


[['ربي يقدر الخير', 618],
 ['ya si slim', 585],
 ['ان شاء اله', 530],
 ['ربي يلطف بينا', 485],
 ['très bone aplication', 484],
 ['امين امين امين', 328],
 ['امين يارب العالمين', 286],
 ['cete aplication est', 245],
 ['جدا جدا جدا', 234],
 ['عبد الطيف المكي', 233],
 ['ne fonctione pas', 232],
 ['bravobravo bravobravo bravobravo', 223],
 ['chômage chômage chômage', 192],
 ['الهم امين يارب', 191],
 ['bravo si slim', 189],
 ['اله ونعم الوكيل', 188],
 ['frais de livraison', 187],
 ['ne marche pas', 187],
 ['et bone continuation', 171],
 ['سي عبد الطيف', 167],
 ['les frais de', 166],
 ['ne sont pas', 164],
 ['je ne peux', 159],
 ['je suis très', 154],
 ['إن شاء اله', 141],
 ['une bone aplication', 141],
 ['mot de pase', 139],
 ['je narive pas', 139],
 ['je nai pas', 132],
 ['ربي يحمي تونس', 124],
 ['achats en ligne', 119],
 ['la mise jour', 119],
 ['ne peux pas', 118],
 ['شاء اله ربي', 117],
 ['masit masit masit', 117],
 ['une très bone', 117],
 ['pour le moment', 114],
 ['je suis satisfait',

# 4-grame features extracted from processed_text


In [14]:
quatre_gram =ngrams_top(data['unique_char'],(4,4),n=100)
quatre_gram.values.tolist()


[['امين امين امين امين', 320],
 ['bravobravo bravobravo bravobravo bravobravo', 216],
 ['chômage chômage chômage chômage', 190],
 ['الهم امين يارب العالمين', 171],
 ['les frais de livraison', 122],
 ['masit masit masit masit', 115],
 ['جدا جدا جدا جدا', 113],
 ['je ne peux pas', 108],
 ['ماسط ماسط ماسط ماسط', 102],
 ['ai honte ai honte', 97],
 ['honte ai honte ai', 96],
 ['signaler signaler signaler signaler', 96],
 ['بالتوفيق ان شاء اله', 93],
 ['حسبي اله ونعم الوكيل', 93],
 ['رسول اله صلى اله', 89],
 ['la dernière mise jour', 88],
 ['une très bone aplication', 83],
 ['أستغفر آله وأتوب إليه', 80],
 ['آله وأتوب إليه أستغفر', 76],
 ['وأتوب إليه أستغفر آله', 76],
 ['إليه أستغفر آله وأتوب', 76],
 ['je ne sais pas', 74],
 ['عليكم ورحمة اله وبركاته', 73],
 ['lastamasta lastamasta lastamasta lastamasta', 73],
 ['degage degage degage degage', 72],
 ['veut travailer veut travailer', 71],
 ['ان شاء اله ربي', 70],
 ['travailer veut travailer veut', 70],
 ['صلى اله عليه وسلم', 68],
 ['de vente en

# length of words
No regex but added one liner to identify length of words in a sentence


In [15]:
data['text_length']=data['unique_char'].str.split().map(lambda x: len(x))
data[['unique_char','text_length']].sample(20)


,unique_char,text_length
135972,['mela hala'],2
155456,['chofe lina yhemnich feha'],4
121845,['وين إعانتك متاع الإنتخابات سي نبيل'],6
7915,['oui mahleh jaw kbir'],4
42722,['machoft nes hedha la ritou maset bhim uf'],8
18620,['رب يسترنا ويبعد علينا هالبلا'],5
42218,['تونس في ثوب جديد وقاحة جرائم قوم لوط بنات بد...,22
166059,['kol chay enti matnajemch chay'],5
28651,['ربي يشفي'],2
198842,['bravo elyes'],2


number of words on cleaned corpus

In [16]:
data['text_length'].sum()

1583826

# Length of characters
No regex but added one liner to identify length of words in a sentence



In [17]:
data['char_length']=data['unique_char'].str.len()
data[['unique_char','char_length']].sample(3)

,unique_char,char_length
97864,['la livraison nest pas stable'],32
111222,['تونس حاجتين ي تعمل فال buz ي تحب تقضي شطر شعب'],49
194333,['wahrtek'],11


number of characters on cleaned corpus

In [18]:
data['char_length'].sum()

9333187

In [19]:
data.drop(['text_clean', 'text_length','char_length'], axis=1, inplace=True)

In [20]:
data['text_cleaned'] = data['unique_char'].apply(lambda s: " ".join([word for word in s.split() if word not in noise1gram]))
data.head(5)

,ID,label,pick_unique,unique_char,text_cleaned
0,0,1,['weldin weldikk bravooooo'],['weldin weldik bravo'],['weldin weldik bravo']
1,1,1,['bravo amine j aime'],['bravo amine j aime'],['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني']
3,3,1,['souut w ziiin lee wlh bch f'],['sout w zin le wlh bch f'],['sout zin le wlh bch f']
4,4,1,['anaa'],['ana'],['ana']


# 2,3,4-grmaes after removing inutile words from 1-grmaes

In [21]:
deux_gram =ngrams_top(data['text_cleaned'],(2,2),n=200)
deux_gram.values.tolist()

[['no content', 6819],
 ['bone aplication', 1202],
 ['شاء اله', 992],
 ['slim riahi', 902],
 ['ربي يسترنا', 851],
 ['ربي يلطف', 779],
 ['سيدي الوزير', 760],
 ['يقدر الخير', 749],
 ['très bone', 715],
 ['يارب العالمين', 658],
 ['ربي يقدر', 655],
 ['si slim', 623],
 ['bone continuation', 617],
 ['la livraison', 576],
 ['جدا جدا', 574],
 ['bravo slim', 524],
 ['ربي يستر', 482],
 ['merci jumia', 464],
 ['عبد الطيف', 432],
 ['جميل جدا', 400],
 ['très bien', 382],
 ['امين يارب', 382],
 ['bon courage', 381],
 ['الحجر الصحي', 377],
 ['black friday', 375],
 ['ne fonctione', 371],
 ['ne pas', 363],
 ['très bon', 362],
 ['aplication très', 357],
 ['يلطف بينا', 349],
 ['nest pas', 341],
 ['امين امين', 336],
 ['cete aplication', 328],
 ['très pratique', 319],
 ['le service', 309],
 ['الناس الكل', 289],
 ['ربي معاك', 288],
 ['ne marche', 278],
 ['انشاء اله', 270],
 ['ربي يوفقك', 268],
 ['rabi mak', 268],
 ['nchalah rabi', 267],
 ['marche pas', 267],
 ['حضر التجول', 265],
 ['fonctione pas', 256],
 ['

In [22]:
trois_gram =ngrams_top(data['text_cleaned'],(3,3),n=200)
trois_gram.values.tolist()

[['ربي يقدر الخير', 618],
 ['très bone aplication', 486],
 ['امين امين امين', 328],
 ['ربي يلطف بينا', 309],
 ['امين يارب العالمين', 286],
 ['جدا جدا جدا', 234],
 ['عبد الطيف المكي', 233],
 ['ne fonctione pas', 233],
 ['chômage chômage chômage', 192],
 ['الهم امين يارب', 191],
 ['اله ونعم الوكيل', 188],
 ['ne marche pas', 187],
 ['سي عبد الطيف', 167],
 ['إن شاء اله', 141],
 ['ان شاء اله', 139],
 ['شاء اله ربي', 124],
 ['ربي يحمي تونس', 124],
 ['ne peux pas', 118],
 ['masit masit masit', 117],
 ['ماسط ماسط ماسط', 112],
 ['bone aplication mais', 104],
 ['الدفع عند الاستلام', 102],
 ['بالتوفيق شاء اله', 101],
 ['حسبي اله ونعم', 97],
 ['tres bone aplication', 97],
 ['ne fonctione plus', 97],
 ['ai honte ai', 97],
 ['honte ai honte', 97],
 ['signaler signaler signaler', 97],
 ['آمين يارب العالمين', 89],
 ['رسول اله صلى', 89],
 ['اله صلى اله', 89],
 ['la dernière mise', 88],
 ['بارك اله فيك', 87],
 ['والسلام رسول اله', 87],
 ['ne peut pas', 87],
 ['عليكم ورحمة اله', 83],
 ['المناسب المكان ال

In [23]:
quatre_gram =ngrams_top(data['text_cleaned'],(4,4),n=200)
quatre_gram.values.tolist()

[['امين امين امين امين', 320],
 ['chômage chômage chômage chômage', 190],
 ['الهم امين يارب العالمين', 171],
 ['masit masit masit masit', 115],
 ['جدا جدا جدا جدا', 113],
 ['ماسط ماسط ماسط ماسط', 102],
 ['ai honte ai honte', 97],
 ['honte ai honte ai', 96],
 ['signaler signaler signaler signaler', 96],
 ['حسبي اله ونعم الوكيل', 93],
 ['رسول اله صلى اله', 89],
 ['والسلام رسول اله صلى', 82],
 ['الصلاة والسلام رسول اله', 80],
 ['أستغفر آله وأتوب إليه', 80],
 ['الرجل المناسب المكان المناسب', 77],
 ['آله وأتوب إليه أستغفر', 76],
 ['وأتوب إليه أستغفر آله', 76],
 ['إليه أستغفر آله وأتوب', 76],
 ['عليكم ورحمة اله وبركاته', 73],
 ['lastamasta lastamasta lastamasta lastamasta', 73],
 ['وسلم الصلاة والسلام رسول', 72],
 ['degage degage degage degage', 72],
 ['veut travailer veut travailer', 71],
 ['travailer veut travailer veut', 70],
 ['صلى اله عليه وسلم', 68],
 ['برافو برافو برافو برافو', 66],
 ['hasbouna alah wakil fikom', 66],
 ['alah wakil fikom hasbouna', 66],
 ['wakil fikom hasbouna alah', 

In [24]:
n_gram =ngrams_top(data['text_cleaned'],(8,8),n=200)
n_gram.values.tolist()

[['امين امين امين امين امين امين امين امين', 304],
 ['chômage chômage chômage chômage chômage chômage chômage chômage', 182],
 ['masit masit masit masit masit masit masit masit', 111],
 ['ai honte ai honte ai honte ai honte', 95],
 ['honte ai honte ai honte ai honte ai', 94],
 ['signaler signaler signaler signaler signaler signaler signaler signaler',
  92],
 ['ماسط ماسط ماسط ماسط ماسط ماسط ماسط ماسط', 90],
 ['أستغفر آله وأتوب إليه أستغفر آله وأتوب إليه', 76],
 ['آله وأتوب إليه أستغفر آله وأتوب إليه أستغفر', 72],
 ['وأتوب إليه أستغفر آله وأتوب إليه أستغفر آله', 72],
 ['إليه أستغفر آله وأتوب إليه أستغفر آله وأتوب', 72],
 ['lastamasta lastamasta lastamasta lastamasta lastamasta lastamasta lastamasta lastamasta',
  69],
 ['veut travailer veut travailer veut travailer veut travailer', 69],
 ['travailer veut travailer veut travailer veut travailer veut', 68],
 ['hasbouna alah wakil fikom hasbouna alah wakil fikom', 64],
 ['alah wakil fikom hasbouna alah wakil fikom hasbouna', 64],
 ['wakil 

In [25]:
data['text_length']=data['text_cleaned'].str.split().map(lambda x: len(x))
data[['text_cleaned','text_length']].sample(20)
data['text_length'].sum()

1320171

In [26]:
1583826-1320253

263573

c'est deja pas mal de libérer 263573 mots

In [27]:
def unifier(phrase):
    # specific
    phrase = re.sub(r"trés", "tres", phrase)
    phrase = re.sub(r"très", "tres", phrase)
    phrase = re.sub(r"apli","application",phrase)
    phrase = re.sub(r"lapli","application",phrase)
    phrase = re.sub(r"l'aplication","application",phrase)
    phrase = re.sub(r"l'application","application",phrase)
    phrase = re.sub(r"aplication","application",phrase)
    
    return phrase

data['text_cleaned']=data['text_cleaned'].apply(lambda s: unifier(s))

In [28]:
data

,ID,label,pick_unique,unique_char,text_cleaned,text_length
0,0,1,['weldin weldikk bravooooo'],['weldin weldik bravo'],['weldin weldik bravo'],3
1,1,1,['bravo amine j aime'],['bravo amine j aime'],['bravo amine j aime'],4
2,2,1,['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني'],4
3,3,1,['souut w ziiin lee wlh bch f'],['sout w zin le wlh bch f'],['sout zin le wlh bch f'],6
4,4,1,['anaa'],['ana'],['ana'],1
...,...,...,...,...,...,...
199092,69995,1,['amazing love'],['amazing love'],['amazing love'],2
199093,69996,1,['ena raniiiiiiiiiii eriahi ca bolomètre kleb ...,['ena rani eriahi ca bolomètre kleb leyeme bin...,['ena rani eriahi ca bolomètre kleb leyeme bin...,15
199094,69997,0,['hada maycharafech tounis'],['hada maycharafech tounis'],['hada maycharafech tounis'],3
199095,69998,0,['leken rajlek haw wemcher inteviou'],['leken rajlek haw wemcher inteviou'],['leken rajlek haw wemcher inteviou'],5


In [29]:
data.drop(['pick_unique', 'unique_char','text_length'], axis=1, inplace=True)

In [30]:
data.head()

,ID,label,text_cleaned
0,0,1,['weldin weldik bravo']
1,1,1,['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني']
3,3,1,['sout zin le wlh bch f']
4,4,1,['ana']


In [31]:
#save our clean dataframe into csv file
#data.to_csv ('cleaned_data.csv')
data.rename(index=str, columns={'text_cleaned':'text_clean'},inplace=True)
#save our clean dataframe into csv file
data.head()

,ID,label,text_clean
0,0,1,['weldin weldik bravo']
1,1,1,['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني']
3,3,1,['sout zin le wlh bch f']
4,4,1,['ana']


In [32]:
data.to_csv('cleaned_data.csv',index=False)

In [33]:
data = pd.read_csv('/content/cleaned_data.csv')
data.head()

,ID,label,text_clean
0,0,1,['weldin weldik bravo']
1,1,1,['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني']
3,3,1,['sout zin le wlh bch f']
4,4,1,['ana']
